# Masuyama 氏らの手法に基づく位相復元 (ADMM法とする)
Y. Masuyama, K. Yatabe and Y. Oikawa, "Griffin-Lim like phase recovery via alternating directionmethod of multipliers," IEEE Signal Processing Letters, vol.26, no.1, pp.184--188, Jan. 2019. https://ieeexplore.ieee.org/document/8552369

In [1]:
import numpy as np
from scipy.io import wavfile
import librosa
from pypesq import pesq
from IPython.display import Audio

In [2]:
IN_WAVE_FILE = "in.wav"  # モノラル音声
OUT_WAVE_FILE = "out_admm_gla.wav"  # 復元音声

In [3]:
FRAME_LENGTH = 1024        # フレーム長 (FFTサイズ)
HOP_LENGTH = 80            # フレームのシフト長
ITERATION = 100            # 位相推定の最大繰り返し数
MULTIPLIER = 0.01          # ADMM法の強さを制御; 0.0のときはGriffin-Lim法に一致

In [4]:
# 音声のロード
fs, data = wavfile.read(IN_WAVE_FILE)
data = data.astype(np.float64)

## 振幅スペクトル（位相復元なので手に入るのはこれのみ）

In [5]:
# 振幅スペクトル（位相復元なので手に入るのはこれのみ）
amp_spec = np.abs(
    librosa.core.stft(
        data, n_fft=FRAME_LENGTH, hop_length=HOP_LENGTH, win_length=FRAME_LENGTH
    )
)

In [6]:
# 乱数の種を指定して再現性を保証
np.random.seed(seed=0)

## ADMM法に基づく位相スペクトルの推定

In [7]:
for i in range(ITERATION):
    if i == 0:
        # 初回は乱数で初期化
        phase_spec = np.random.rand(*amp_spec.shape)
        control_spec = np.zeros(amp_spec.shape)
    else:
        # 振幅スペクトルと推定された位相スペクトルから複素スペクトログラムを復元
        recovered_spec = amp_spec * np.exp(1j * phase_spec)

        # 短時間フーリエ逆変換で音声を復元
        combined = recovered_spec + control_spec
        recovered = librosa.core.istft(
            combined, hop_length=HOP_LENGTH, win_length=FRAME_LENGTH
        )

        # 復元音声から複素スペクトログラムを再計算
        complex_spec = librosa.core.stft(
            recovered,
            n_fft=FRAME_LENGTH,
            hop_length=HOP_LENGTH,
            win_length=FRAME_LENGTH,
        )
        complex_spec = MULTIPLIER * combined + complex_spec
        complex_spec /= 1.0 + MULTIPLIER

        # 初回以降は計算済みの複素スペクトログラムから位相スペクトルを推定
        control_spec = control_spec + recovered_spec - complex_spec
        phase_spec = np.angle(complex_spec - control_spec)

In [8]:
# 音声を復元
recovered_spec = amp_spec * np.exp(1j * phase_spec)
recovered_admm = librosa.core.istft(
    recovered_spec, hop_length=HOP_LENGTH, win_length=FRAME_LENGTH
)
# pesqで音質を評価
print("PESQ = ", pesq(recovered_admm, data[: len(recovered)], fs))

PESQ =  4.314631462097168


## 比較のため、Griffin-Lim法による位相復元
D. W. Griffin and J. S. Lim, “Signal estimation from modified short-time Fourier transform,” IEEE Trans. ASSP, vol.32, no.2, pp.236–243, Apr. 1984.
https://ieeexplore.ieee.org/document/1164317

In [9]:
# Griffin-Lim法に基づく位相スペクトルの推定
for i in range(ITERATION):
    if i == 0:
        # 初回は乱数で初期化
        phase_spec = np.random.rand(*amp_spec.shape)
    else:
        # 振幅スペクトルと推定された位相スペクトルから複素スペクトログラムを復元
        recovered_spec = amp_spec * np.exp(1j * phase_spec)

        # 短時間フーリエ逆変換で音声を復元
        recovered = librosa.core.istft(recovered_spec, hop_length=HOP_LENGTH,
                                       win_length=FRAME_LENGTH)

        # 復元音声から複素スペクトログラムを再計算
        complex_spec = librosa.core.stft(recovered, n_fft=FRAME_LENGTH,
                                         hop_length=HOP_LENGTH,
                                         win_length=FRAME_LENGTH)

        # 初回以降は計算済みの複素スペクトログラムから位相スペクトルを推定
        phase_spec = np.angle(complex_spec)

In [10]:
# 音声を復元
recovered_spec = amp_spec * np.exp(1j * phase_spec)
recovered_gla = librosa.core.istft(recovered_spec, hop_length=HOP_LENGTH,
                               win_length=FRAME_LENGTH)
# pesqで音質を評価
print("PESQ = ", pesq(recovered_gla, data[: len(recovered_gla)], fs))

PESQ =  4.141395568847656


## 比較のため、Fast Griffin-Lim法による位相復元
Perraudin, N., Balazs, P., & Søndergaard, P. L. “A fast Griffin-Lim algorithm,” IEEE Workshop on Applications of Signal Processing to Audio and Acoustics (pp. 1-4), Oct. 2013.
https://ieeexplore.ieee.org/document/6701851

In [11]:
# 比較のため、Griffin-Lim法により位相復元
recovered_fgla = librosa.griffinlim(
    amp_spec, n_iter=ITERATION, hop_length=HOP_LENGTH, random_state=0
)
# pesqで音質を評価
print("PESQ = ", pesq(recovered_fgla, data[: len(recovered_fgla)], fs))

PESQ =  4.310971736907959


## 音声の聴き比べ

### 元音声

In [12]:
Audio(data, rate=fs)

### ADMM法による復元

In [13]:
Audio(recovered_admm, rate=fs)

### 従来のGriffin-Lim法による復元

In [14]:
Audio(recovered_gla, rate=fs)

### Fast Griffin-Lim法による復元

In [15]:
Audio(recovered_fgla, rate=fs)